In [2]:
# pip install rapidfuzz

In [27]:
import pandas as pd 
import numpy as np
import os
from sklearn.neighbors import NearestNeighbors
from rapidfuzz import process

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Client\.cache\kagglehub\datasets\rounakbanik\the-movies-dataset\versions\7


In [5]:
files = os.listdir(path)
files

['credits.csv',
 'keywords.csv',
 'links.csv',
 'links_small.csv',
 'movies_metadata.csv',
 'ratings.csv',
 'ratings_small.csv']

In [6]:
user_ratings = pd.read_csv(os.path.join(path,files[5]))
user_ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [7]:
#drop timestamp since no use
user_ratings.drop(columns='timestamp',inplace=True)
user_ratings = user_ratings.groupby(['userId','movieId'])['rating'].mean().reset_index()
user_ratings

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [8]:
movie_metadata = pd.read_csv(os.path.join(path,files[4]))
movie_metadata = movie_metadata.rename(columns={'id':'movieId'})
movie_metadata = movie_metadata[movie_metadata['movieId'].str.isnumeric()]
movie_metadata['movieId'] = movie_metadata['movieId'].astype(int)
movie_metadata.head()

C:\Users\Client\AppData\Local\Temp\ipykernel_13596\3670185964.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_metadata = pd.read_csv(os.path.join(path,files[4]))


,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
movie_metadata = movie_metadata[['movieId','title','genres']]
movie_metadata.head()

,movieId,title,genres
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"


In [12]:
movie_data = user_ratings.merge(movie_metadata, on='movieId')
movie_data

,userId,movieId,rating,title,genres
0,1,110,1.0,Three Colors: Red,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na..."
1,1,147,4.5,The 400 Blows,"[{'id': 18, 'name': 'Drama'}]"
2,1,858,5.0,Sleepless in Seattle,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
3,1,1246,5.0,Rocky Balboa,"[{'id': 18, 'name': 'Drama'}]"
4,1,1968,4.0,Fools Rush In,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name..."
...,...,...,...,...,...
11437632,270896,48780,5.0,Boat,"[{'id': 10769, 'name': 'Foreign'}]"
11437633,270896,49530,4.0,In Time,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam..."
11437634,270896,54001,4.0,The Traveler,"[{'id': 18, 'name': 'Drama'}]"
11437635,270896,54503,4.0,The Mystery of Chess Boxing,"[{'id': 28, 'name': 'Action'}, {'id': 10769, '..."


In [13]:
size = 100000
movie_data = movie_data.sample(n =size,replace= False,random_state=456)

user_item_matrix = movie_data.pivot_table(index=['movieId'],columns=['userId'],values=['rating'],aggfunc='mean').fillna(0)
user_item_matrix

rating                                                                 \
userId  2      7      8      19     24     27     34     43     49     58       
movieId                                                                         
2          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
11         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
170263     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
170315     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
171759     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
173557     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
173891     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

         ...                                                                 \
userId   ... 270869 270871 270876 270877 270879 270885 270886 270887 270894   
movieId  ...                                                                  
2        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
11       ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
170263   ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
170315   ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
171759   ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
173557   ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
173891   ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                
userId  270896  
movieId         
2          0.0  
3          0.0  
5          0.0  
6          0.0  
11         0.0  
...        ...  
170263     0.0  
170315     0.0  
171759     0.0  
173557     0.0  
173891     0.0  

[3500 rows x 61712 columns]

In [15]:
knn_model = NearestNeighbors(metric = 'cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
knn_model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [22]:
def recommendor(movie_name,matrix,model,n_recs,movie_data):
    best_match = process.extractOne(movie_name, movie_data['title'])
    if best_match is None:
        return pd.DataFrame([{'Title':f'No match found for {movie_name}'}])

    matched_title = best_match[0]
    movie_id = movie_data.loc[movie_data['title'] == matched_title, 'movieId'].iloc[0]

    if movie_id not in matrix.index:
        return pd.DataFrame([{'Title':f'movie {matched_title} not in matrix'}])
    movie_idx = matrix.index.get_loc(movie_id)
    
    distances,indices = knn_model.kneighbors(user_item_matrix.iloc[movie_idx].values.reshape(1,-1), n_neighbors=n_recs+1)
    
    rec_ids = indices.flatten()[1:]
    rec_movies = matrix.index[rec_ids]
    
    recs = (
        movie_data[movie_data['movieId'].isin(rec_movies)]
        .drop_duplicates('movieId')[['title']]
        .reset_index(drop = True)
    )
    
    return recs


In [26]:
n_recs = 10
recommendations = recommendor('Titanic',user_item_matrix,knn_model,n_recs,movie_data)
print(recommendations)

                             title
0            The French Revolution
1         The Day the Sky Exploded
2                           Tulpan
3                  Run, Simon, Run
4              Beach Blanket Bingo
5                          Harvest
6               Ten Little Indians
7  Ingmar Bergman on Life and Work
8                    The Collector
9               A Woman Is a Woman
